# Analysis of LSOA results

In [1]:
file_name = 'lsoa_base';
limit_to_england = False 

## Set analysis parameters

In [2]:
stroke_type_propotions = {
    'haemorragic': 0.136,
    'lvo': 0.33 * (1 - 0.136),
    'nlvo': 0.67 * (1 - 0.136)}

In [3]:
target_ivt = 0.2
target_mt = 0.1
prop_mt_previously_receiving_ivt = 0.85

prop_lvo_receiving_mt = target_mt / stroke_type_propotions['lvo']
print(f'{prop_lvo_receiving_mt:0.3f}')

0.351


In [4]:
prop_lvo_receiving_ivt = prop_lvo_receiving_mt * prop_mt_previously_receiving_ivt
print(f'{prop_lvo_receiving_ivt:0.3f}')

0.298


In [5]:
prop_nlvo_receiving_ivt = (
    (target_ivt - (prop_lvo_receiving_ivt * stroke_type_propotions['lvo'])) / 
    stroke_type_propotions['nlvo'])

print(f'{prop_nlvo_receiving_ivt:0.3f}')

0.199


In [6]:
# Check sums
prop_all_receiving_ivt = (
    (stroke_type_propotions['nlvo'] * prop_nlvo_receiving_ivt) +
    (stroke_type_propotions['lvo'] * prop_lvo_receiving_ivt))

print(f'{prop_all_receiving_ivt:0.3f}')

0.200


## Import packages

In [7]:
import contextily as ctx
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_rows', 150)

### Load LSOA model output data

In [8]:
lsoa_data = pd.read_csv(f'./output/{file_name}.csv')
lsoa_data.head(3).T

,0,1,2
lsoa,Adur 001A,Adur 001B,Adur 001C
closest_ivt_unit,BN25BE,BN25BE,BN112DH
closest_ivt_time,17.6,18.7,17.6
closest_mt_unit,BN25BE,BN25BE,BN25BE
closest_mt_time,17.6,18.7,19.8
transfer_mt_unit,BN25BE,BN25BE,BN25BE
transfer_mt_time,0.0,0.0,31.6
mt_transfer_required,False,False,True
msu_unit,BN25BE,BN25BE,BN25BE
msu_time,17.6,18.7,19.8
